In [1]:
from tmilib import *

In [2]:
users_with_tab_focus_data = []

for user in list_users():
  if not sdir_exists('tab_focus_times_for_user/' + user + '.json'):
    continue
  tab_focus_times = get_tab_focus_times_for_user(user)
  if len(tab_focus_times) > 1000:
    users_with_tab_focus_data.append(user)

print len(users_with_tab_focus_data)

65


In [3]:
user = users_with_tab_focus_data[0]
print user

#url_to_visits = get_history_visits_for_user(user)

dwCCD1uofp


In [4]:
tab_focus_times = get_tab_focus_times_for_user(user)
print len(tab_focus_times)
#print tab_focus_times

5061


In [5]:
ordered_visits = get_history_ordered_visits_for_user(user)
#first_site = url_to_visits.keys()[0]
#print url_to_visits[first_site]

In [6]:
class ReconstructFocusTimes:
  def __init__(self):
    self.output = []
    self.curitem = {}
  def get_output(self):
    self.end_input()
    return self.output
  def end_input(self):
    if 'url' in self.curitem:
      last_active = self.curitem['active']
      self.curitem['end'] = last_active + 60*1000
      self.output.append(self.curitem)
      self.curitem = {}
  def end_session(self, curtime):
    if 'url' in self.curitem:
      last_active = self.curitem['active']
      # ensures that end < last_active+60 seconds
      self.curitem['end'] = min(curtime, last_active + 60*1000)
      self.output.append(self.curitem)
      self.curitem = {}
  def start_session(self, url, curtime):
    if url == None:
      raise Exception('start_session should not be called with url==None')
    self.end_session(curtime)
    # start: first event in the session
    # active: last event which was active in the session
    # end: when we believe the session ended
    self.curitem = {'url': url, 'start': curtime, 'active': curtime}
  def continue_session(self, url, curtime):
    if url == None:
      raise Exception('continue_session should not be called with url==None')
    if 'url' not in self.curitem:
      self.start_session(url, curtime)
      return
    prevurl = self.curitem['url']
    if url == prevurl: # still on same site
      # has it been less than 60 seconds since last activity?
      prev_active_time = self.curitem['active']
      if curtime < prev_active_time + 60*1000:
        self.curitem['active'] = curtime
        return
    # have gone to different site
    self.end_session(curtime)
    self.start_session(url, curtime)
  def process_history_line(self, data):
    url = data['url']
    curtime = data['visitTime']
    self.continue_session(url, curtime)

def to_visit_lengths(ordered_visits):
  focus_time_reconstructor = ReconstructFocusTimes()
  for visit in ordered_visits:
    focus_time_reconstructor.process_history_line(visit)
  return focus_time_reconstructor.get_output()

reconstructed_visit_lengths = to_visit_lengths(ordered_visits)

In [7]:
print len(reconstructed_visit_lengths)

19896


In [8]:
def get_earliest_start_time(visit_lengths):
  if len(visit_lengths) < 1:
    raise Exception('get_earliest_time called with empty list')
  return visit_lengths[0]['start']

def get_last_end_time(visit_lengths):
  if len(visit_lengths) < 1:
    raise Exception('get_last_visit_time called with empty list')
  return visit_lengths[-1]['end']

print get_earliest_start_time(reconstructed_visit_lengths)
print get_earliest_start_time(tab_focus_times)

print get_last_end_time(reconstructed_visit_lengths)
print get_last_end_time(tab_focus_times)

1.45416431647e+12
1.45813152485e+12
1.45823578095e+12
1.45878745735e+12


In [21]:
def ignore_all_before_start_or_after_end(visit_lengths, start_time, end_time):
  output = []
  for x in visit_lengths:
    if x['start'] < start_time - 1000: # 1 second before or after
      continue
    if x['end'] > end_time + 1000:
      continue
    output.append(x)
  return output

ref_start_time = get_earliest_start_time(tab_focus_times) + 1000*3600*10
#ref_end_time = get_last_end_time(tab_focus_times)
ref_end_time = ref_start_time + 1000*3600*11

evaluated_reconstructed_visits = ignore_all_before_start_or_after_end(reconstructed_visit_lengths, ref_start_time, ref_end_time)
evaluated_tab_focus_times = ignore_all_before_start_or_after_end(tab_focus_times, ref_start_time, ref_end_time)

In [10]:
print ref_start_time

1.45813152485e+12


In [11]:
print evaluated_tab_focus_times[0]

{u'url': u'https://tmi.stanford.edu/mturk3.html?username=A2JCK494NV7TFX', u'start': 1458131524850.0, u'end': 1458131545126.0, u'active': 1458131535855.0}


In [19]:
for x in reconstructed_visit_lengths:
  url = x['url']
  if 'tmi.stanford.edu' in url:
    print x
    break

{'url': u'https://tmi.stanford.edu/mturk3.html?username=A2JCK494NV7TFX', 'start': 1458131501635.586, 'end': 1458131580974.248, 'active': 1458131520974.248}


In [22]:
print len(evaluated_reconstructed_visits)
print len(evaluated_tab_focus_times)

58
106


In [25]:
#for x in evaluated_reconstructed_visits:
#  print x

In [24]:
#for x in evaluated_tab_focus_times:
#  print x

{u'url': u'http://cmu.qualtrics.com/jfe/form/SV_3qRBhwnc3kYQRhj?UID=81254&XCL=', u'start': 1458167629462.0, u'end': 1458167718310.0, u'active': 1458167650714.0}
{u'url': u'http://cmu.qualtrics.com/jfe/form/SV_3qRBhwnc3kYQRhj?UID=81254&XCL=', u'start': 1458167718310.0, u'end': 1458167846951.0, u'active': 1458167846266.0}
{u'url': u'https://www.mturk.com/mturk/findhits?match=false&hit_scraper#', u'start': 1458167846951.0, u'end': 1458167848231.0, u'active': 1458167846951.0}
{u'url': u'https://www.mturk.com/mturk/previewandaccept?groupId=3SI493PTTXLLJABRFNCMBK7DOIEZD8', u'start': 1458167848231.0, u'end': 1458167857291.0, u'active': 1458167850250.0}
{u'url': u'https://www.mturk.com/mturk/return?requesterId=&canAccept=&hitId=3Y5140Z9DYBAU1PPSPAMQLWBZL9IPW&externalHit=true&groupId=3SI493PTTXLLJABRFNCMBK7DOIEZD8', u'start': 1458167857291.0, u'end': 1458167858426.0, u'active': 1458167858302.0}
{u'url': u'https://www.mturk.com/mturk/findhits?match=false&hit_scraper#', u'start': 1458167858426.0,

In [16]:
for x in reconstructed_visit_lengths:
  url = x['url']
  if url == 'https://www.mturk.com/mturk/continue?hitId=3ZY8KE4ISJ42CAV8NVJ18F1PI9SQVJ':
    print x

{'url': u'https://www.mturk.com/mturk/continue?hitId=3ZY8KE4ISJ42CAV8NVJ18F1PI9SQVJ', 'start': 1458131499396.9, 'end': 1458131501635.586, 'active': 1458131499396.9}


In [17]:
for x in tab_focus_times:
  url = x['url']
  if url == 'https://www.mturk.com/mturk/continue?hitId=3ZY8KE4ISJ42CAV8NVJ18F1PI9SQVJ':
    print x

{u'url': u'https://www.mturk.com/mturk/continue?hitId=3ZY8KE4ISJ42CAV8NVJ18F1PI9SQVJ', u'start': 1458131651814.0, u'end': 1458131658442.0, u'active': 1458131651824.0}
{u'url': u'https://www.mturk.com/mturk/continue?hitId=3ZY8KE4ISJ42CAV8NVJ18F1PI9SQVJ', u'start': 1458131663976.0, u'end': 1458131666447.0, u'active': 1458131666054.0}


In [16]:

'''
def to_visit_lengths(ordered_visits):
  output = []
  for item,idx in ordered_visits:
    url = item['url']
    visit_time = item['visitTime'] # timestamp in milliseconds
    result = {'url': url, 'start': visit_time, 'active': visit_time}
    visit_duration = 30*1000 # 30 seconds in milliseconds
    if idx < len(ordered_visits):
      next_item = ordered_visits[idx+1]
      next_visit_time = next_item['visitTime']
      time_until_next_visit = next_visit_time - visit_time
      if visit_duration > time_until_next_visit: # more than 30 minutes until next visit
        
      visit_duration = min(visit_duration, time_until_next_visit)
    output.append({'url': url, 'start': visit_time, })
'''

"\ndef to_visit_lengths(ordered_visits):\n  output = []\n  for item,idx in ordered_visits:\n    url = item['url']\n    visit_time = item['visitTime'] # timestamp in milliseconds\n    result = {'url': url, 'start': visit_time, 'active': visit_time}\n    visit_duration = 30*1000 # 30 seconds in milliseconds\n    if idx < len(ordered_visits):\n      next_item = ordered_visits[idx+1]\n      next_visit_time = next_item['visitTime']\n      time_until_next_visit = next_visit_time - visit_time\n      if visit_duration > time_until_next_visit: # more than 30 minutes until next visit\n        \n      visit_duration = min(visit_duration, time_until_next_visit)\n    output.append({'url': url, 'start': visit_time, })\n"